### Only best guideline

In [ ]:
from config import GUIDELINES

all_guidelines = "\n".join(["- " + guideline for guideline in GUIDELINES])

extract_template = """You are a radiologist expert. Do not make up additional information.
=========
TASK: You are given a PATIENT PROFILE. You need to perform the following information referencing from the PATIENT PROFILE:
1. Extract relevant information for recommendation of imaging procedure, including age, symptomps, previous diagnosis, stage of diagnosis (INITIAL IMAGING OR NEXT STUDY) and suspected conditions, if any.
Only return information given inside the PROFILE, do not make up other information.
2. Return the best guideline from the following list of guidelines that is relevant to the recommendations of imaging procedure given patient profile. If there are no relevant guideline, return empty string.
The recommended guideline must match the exact text given in the list.
{}
=========
OUTPUT INSTRUCTION:
Output your answer as follow:
1. Relevant information:
2. Relevant guidelines:
=========
""".format(all_guidelines)

human_template = "PATIENT PROFILE: {query_str}"


from langchain.prompts import ChatPromptTemplate as LCChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

EXTRACT_TEMPLATE = LCChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(extract_template),
        HumanMessagePromptTemplate.from_template(human_template)
    ]
)

extract_chain = LLMChain(
    prompt=EXTRACT_TEMPLATE,
    llm=ChatOpenAI(model="gpt-4", temperature=0, max_tokens=512)
    )

In [ ]:
best_guideline_answers = []
for clinical_file in tqdm(testcase_df["Clinical File"],
                          total=len(testcase_df["Clinical File"])):
    profile = remove_final_sentence(clinical_file)
    response = extract_chain(profile)
    best_guideline_answers.append(response["text"])

100%|██████████| 71/71 [24:14<00:00, 20.49s/it]   


In [ ]:
profiles, guidelines = [], []

for answer in best_guideline_answers:
    if answer.endswith("."):
        answer = answer[:-1]
    pattern = r"1. Relevant information:([\S\s]+)2. Relevant guidelines:([\S\s]*)"
    profile, guidelines_str = re.findall(pattern, answer)[0]
    
    guidelines_str = guidelines_str.replace("- ", "")
    guidelines_str = guidelines_str.strip()
    guidelines_str = guidelines_str.replace("\n", ", ")
    
    extracted_guidelines = re.findall(r"([A-Za-z ]+)", guidelines_str)
    for i, extracted_guideline in enumerate(extracted_guidelines):
        extracted_guideline = extracted_guideline.lower()
        min_dist, nearest_text = calculate_min_dist(extracted_guideline, GUIDELINES, True)
        if min_dist <= 1:
            extracted_guidelines[i] = nearest_text
        else:
            print(extracted_guideline, nearest_text)
    
    profiles.append(profile)    
    guidelines.append(extracted_guidelines)

In [ ]:
extracted_info_best_json = {"profiles": profiles, "guidelines": guidelines}

with open(os.path.join(ARTIFACT_DIR, "extracted_best.json"), "w") as f:
    json.dump(extracted_info_best_json, f)